In [ ]:
!pip install talos
import talos
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend
from tensorflow.keras.activations import softmax, sigmoid, relu
from tensorflow.keras.optimizers import Adam
import pickle
from talos import Scan, Evaluate, Deploy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
with open('train', 'rb') as file:
    train_dict = pickle.load(file, encoding='bytes')

with open('test', 'rb') as file:
    test_dict = pickle.load(file, encoding='bytes')

X_train = train_dict[b'data']
y_train = train_dict[b'coarse_labels']

X_test = test_dict[b'data']
y_test = test_dict[b'coarse_labels']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

y_train = tf.keras.utils.to_categorical(y_train, 100)
y_test = tf.keras.utils.to_categorical(y_test, 100)

p = {
    'units': [120, 240],
    'hidden_activations': ['relu', 'sigmoid'],
    'activation': ['softmax', 'sigmoid'],
    'loss': ['mse', 'categorical_crossentropy'],
    'optimizer': ['adam', 'adagrad'],
    'batch_size': [1000, 2000]
}

def create_model(X_train,y_train,X_test,y_test,params):
  model = Sequential()
  model.add(Dense(units=params['units'], input_shape = X_train.shape[1:]))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=params['units'], activation=params['hidden_activations']))
  model.add(Dense(units=100, activation=params['activation']))

  model.compile(loss=params['loss'], optimizer=params['optimizer'], metrics=['accuracy'])

  out = model.fit(x=X_train, y=y_train, batch_size=params['batch_size'], epochs=200)

  return out,model

t = talos.Scan(x=X_train, y=y_train, params=p, model=create_model, x_val=X_test, y_val=y_test, experiment_name="talos_output")
#e = talos.Evaluate(t)



  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 1/200
50/50 [==============================] - 5s 15ms/step - loss: 0.0096 - accuracy: 0.0688
Epoch 2/200
50/50 [==============================] - 1s 15ms/step - loss: 0.0090 - accuracy: 0.1677
Epoch 3/200
50/50 [==============================] - 1s 16ms/step - loss: 0.0087 - accuracy: 0.2183
Epoch 4/200
50/50 [==============================] - 1s 14ms/step - loss: 0.0086 - accuracy: 0.2490
Epoch 5/200
50/50 [==============================] - 1s 14ms/step - loss: 0.0084 - accuracy: 0.2688
Epoch 6/200
39/50 [======================>.......] - ETA: 0s - loss: 0.0083 - accuracy: 0.2773

KeyboardInterrupt: ignored

In [ ]:
#t = talos.Scan(x=X_train, y=y_train, params=p, model=create_model, x_val=X_test, y_val=y_test, experiment_name="talos_output")
e = talos.Evaluate(t)
#print("Evaluation")
#result = model.evaluate(X_test, y_test)
evaluation = e.evaluate(X_test, y_test, metric = 'accuracy', task = 'multi_label')
print("Max Accuracy:")
print(max(evaluation))
print("All Accuracies:")
print(evaluation[0])
print(evaluation[1])
print(evaluation[2])
print(evaluation[3])
print(evaluation[4])
model = Deploy(t, model_name = 'CIFAR100', metric = 'accuracy')
#y_pred  = model.model.predict(X_test)

NameError: ignored